# Simple ZMQ Loopback - Software Static Gain


In [ ]:
import zmq
import sys
import numpy as np
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import threading

## Create a PUSH Socket

In [ ]:
_PROTOCOL = "tcp://"
_SERVER = "192.168.1.119"          # localhost
_PUSH_PORT = ":20000"
_PUSH_ADDR = _PROTOCOL + _SERVER + _PUSH_PORT
push_context = zmq.Context()
push_sock = push_context.socket (zmq.PUSH)
rc = push_sock.bind (_PUSH_ADDR)

## Create a PULL Socket

In [ ]:
_PROTOCOL = "tcp://"
_SERVER = "192.168.1.135"          # localhost
_PULL_PORT = ":10000"
_PULL_ADDR = _PROTOCOL + _SERVER + _PULL_PORT
pull_context = zmq.Context()
pull_sock = pull_context.socket (zmq.PULL)
rc = pull_sock.connect (_PULL_ADDR)

## Read Data, Gain Signal, then Send Data

In [ ]:
slider = widgets.FloatSlider(min=0.1,max=10,value=1,description='Signal Gain:')
display(slider)


def slider_handler(change):
    slider.value = change.new
    
slider.observe(slider_handler, names='value')

def work(slider):
    while True:
        data = pull_sock.recv()
        data = np.frombuffer(data, dtype=np.complex64, count=-1)
        push_sock.send(data*slider.value)
        
    
thread = threading.Thread(target=work, args=(slider,))
thread.start()